<a href="https://colab.research.google.com/github/Lakshya11223/Lang_Chain_LLM/blob/main/YT_summariser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
os.environ["GOOGLE_API_KEY"] = ""

In [4]:
!pip install -q youtube-transcript-api langchain-community \
               google-generativeai langchain-google-genai \
               faiss-cpu tiktoken python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.2/485.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [7]:
!pip uninstall -y langchain-community
!pip install langchain-community --no-deps
!pip install requests==2.32.4


Found existing installation: langchain-community 0.4.1
Uninstalling langchain-community-0.4.1:
  Successfully uninstalled langchain-community-0.4.1
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
Using cached langchain_community-0.4.1-py3-none-any.whl (2.5 MB)


In [26]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [35]:
video_id = "YyCXp_E4sMU"
try:
    ytt_api = YouTubeTranscriptApi()
    transcript_list = ytt_api.fetch(video_id, languages=["en"])

    transcript = " ".join([val.text for val in transcript_list])
    print(transcript)
except TranscriptsDisabled:
  print("Transcript disabled for this video")


A very good evening to you ladies and gentlemen. We can do better. A very good evening to you ladies and gentlemen. Oh wow. It's an absolute joy, honor and pleasure to be here this evening. Let me kick this evening off with a favorite story of mine. There was a little boy about 6 years old who came to a balloon seller who was selling balloons on the street and he asked the balloon seller, "Uncle, will this red balloon go up in the air?" The balloon seller said, "Of course it will." Said, "And the blue one?" 100% it will. The pink one, most certainly it will. And you know how kids can be. They can ask funny questions and I give funny answers as well. A 4-year-old once came to a shopkeeper and said to the shopkeeper girl, he said, "Uncle, fouryear-old girl." The shopkeeper said, he said uncle. The shopkeeper said, he said uncle children very innocent, charming, clever, smart. They can ask funny questions and they can give funny answers too. A teacher in class asked a six-year-old which s

In [36]:
transcript

'A very good evening to you ladies and gentlemen. We can do better. A very good evening to you ladies and gentlemen. Oh wow. It\'s an absolute joy, honor and pleasure to be here this evening. Let me kick this evening off with a favorite story of mine. There was a little boy about 6 years old who came to a balloon seller who was selling balloons on the street and he asked the balloon seller, "Uncle, will this red balloon go up in the air?" The balloon seller said, "Of course it will." Said, "And the blue one?" 100% it will. The pink one, most certainly it will. And you know how kids can be. They can ask funny questions and I give funny answers as well. A 4-year-old once came to a shopkeeper and said to the shopkeeper girl, he said, "Uncle, fouryear-old girl." The shopkeeper said, he said uncle. The shopkeeper said, he said uncle children very innocent, charming, clever, smart. They can ask funny questions and they can give funny answers too. A teacher in class asked a six-year-old which

## Indexing

In [39]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
chunks = splitter.create_documents([transcript])

In [40]:
len(chunks)

19

## Embeddings generation

In [41]:
embedding = GoogleGenerativeAIEmbeddings(model='gemini-embedding-001')
vector = FAISS.from_documents(chunks, embedding)

In [42]:
vector.index_to_docstore_id

{0: '25af7904-36aa-401d-ae47-a0aee5685e65',
 1: '262269c1-8d24-4169-92a8-71919f60fde7',
 2: 'fed0ebf8-c2ee-445d-aed8-812349bb249e',
 3: '4fa549b5-46be-4eb3-927f-4d45bc77881f',
 4: '4f631d06-5517-4893-a3df-2d2d1265bdde',
 5: '2d9c4a31-ed36-49f1-baa8-5d5ae179928e',
 6: 'd0fc54e5-695f-440c-ad9f-b09a1acc0886',
 7: 'e70ba5b6-80d8-4501-a9c0-923db2ff9280',
 8: '906001de-cb0f-4929-8dd0-41cb56f8c340',
 9: '935f5d01-03df-4a22-9e8f-bb413e563e0f',
 10: '993573f5-4fb6-48e1-8e4c-375d9622f128',
 11: 'b5c1ccd3-c13a-4e2a-b50a-ace4286258bc',
 12: '0517f410-f704-4b92-8e35-8653aea3cfa4',
 13: 'f456c7a1-f1f9-412d-bdfb-41c1837c320e',
 14: 'c7763346-de0b-480e-a51e-e79c958fc084',
 15: 'b1643701-7a3f-4fc9-8ee8-74b37c24ed56',
 16: '29c454b9-63d4-4e8b-b0b8-abaaf4411d8f',
 17: '3772bca0-14c3-47d0-8665-ed186b26786f',
 18: 'c54c032f-3221-4e12-a165-01e5fc476185'}

## Retriever

In [43]:
retriever = vector.as_retriever(search_type="similarity",search_kwargs={"k":2})

In [45]:
LLM = ChatGoogleGenerativeAI(model='gemini-3-flash-preview',temperature= 0.2)

In [80]:
prompt = PromptTemplate(
    template = """You are here to give the answer of user prompt , you have to give the answer strictly on the basis of content which provided to you
     Please do not over-explain give short answer if you do not have knowledge just I don't no
                  Question : {question}
                  Content : {content}
               """,
    input_variables = ["question","content"]
)

In [81]:
context = retriever.invoke("What is the age of boy come come to balloon seller")

In [82]:
question = "What is the age of boy come come to balloon seller"

In [85]:
fprompt = prompt.invoke({
    "question": question,
    "content": context
})

In [86]:
Answer = LLM.invoke(fprompt)
Answer.text

'About 6 years old.'

## Chain

In [87]:
from langchain_core.runnables import RunnableParallel,RunnablePassthrough

In [88]:
data = RunnableParallel(
    question = RunnablePassthrough(),
    content =  lambda x : retriever.invoke(question)
)

In [89]:
chain = data | prompt | LLM

In [90]:
ans = chain.invoke("What is the age of boy come come to balloon seller provide integer answer")
ans.text

'6'